In [1]:
# Monitor SC

In [2]:
import logging
import uuid

import smartpynector as sp
from utils import *

In [3]:
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

In [4]:
# Constants
READ_URL = "https://graphdb.odissei.nl/repositories/MateuszTest"
WRITE_URL = "https://graphdb.odissei.nl/repositories/MateuszTest/statements"

THERMOSTAT_API_URL = "http://0.0.0.0:8001/thermostat"
PREFIXES = {
    "rdf": "http://www.w3.org/1999/02/22-rdf-syntax-ns#",
    "saref": "https://w3id.org/saref#",
}

MEAS_GRAPH_PATTERN = """?meas rdf:type saref:Measurement .
                     ?meas saref:hasValue ?temp .
                     ?meas saref:isMeasuredIn saref:TemperatureUnit .
                     ?meas saref:hasTimestamp ?timestamp .
                     ?meas saref:isMeasurementOf ?room_id .
                     ?meas saref:relatesToProperty saref:Temperature .
                     ?meas saref:measurementMadeBy ?device_id ."""

COMMAND_GRAPH_PATTERN = """?command rdf:type saref:GetMeterHistoryCommand .
                        ?command saref:isCommandOf ?device_id .
                        ?command saref:hasTimestampStart ?startTimestamp .
                        ?command saref:hasTimestampEnd ?endTimestamp .
                        ?command saref:relatesToProperty saref:Temperature ."""

In [5]:
def start_monitor_kb(kb_id, kb_name, kb_description, ke_endpoint):
    register_knowledge_base(kb_id, kb_name, kb_description, ke_endpoint)

    global ask_measurements_ki
    ask_measurements_ki = register_ask_knowledge_interaction(
        COMMAND_GRAPH_PATTERN,
        "ask-measurements",
        kb_id,
        ke_endpoint,
        PREFIXES,
    )

In [6]:
start_monitor_kb(
    "http://example.org/monitor",
    "Monitor",
    "Monitor for storage",
    "http://knowledge_engine:8280/rest/",
)

2023-06-20 01:28:04 INFO registered Monitor
2023-06-20 01:28:04 INFO received issued knowledge interaction id: http://example.org/monitor/interaction/ask-measurements


In [7]:
def graph_data(start_time, end_time):
    historical_measurements = ask(
        [
            {
                "command": f"<{THERMOSTAT_API_URL}/command/{str(uuid.uuid4())}>",
                "device_id": f'"{THERMOSTAT_API_URL}/devices/1"',
                "startTimestamp": f"{start_time}",
                "endTimestamp": f"{end_time}",
            }
        ],
        ask_measurements_ki,
        "http://example.org/monitor",
        "http://knowledge_engine:8280/rest/",
    )

    print(historical_measurements)
    return historical_measurements

In [8]:
from datetime import datetime

from IPython.display import display
from ipywidgets import Button, DatetimePicker, HBox


def convert_datetimes(start_widget, end_widget):
    start_datetime_obj = start_widget.value.astimezone().replace(microsecond=0)
    end_datetime_obj = end_widget.value.astimezone().replace(microsecond=0)
    return start_datetime_obj, end_datetime_obj


start_widget = DatetimePicker(description="Start Time:", disabled=False)
end_widget = DatetimePicker(description="End Time:", disabled=False)
submit_button = Button(description="Submit", button_style="danger")
display(HBox([start_widget, end_widget, submit_button]))


def on_button_click(button):
    start_timestamp, end_timestamp = convert_datetimes(start_widget, end_widget)
    logger.info(f"Start: {start_timestamp}, End: {end_timestamp}")
    graph_data(start_timestamp, end_timestamp)


submit_button.on_click(on_button_click)